# Phase III: First ML Proof of Concept (5\%)

### Team Names:
- Melanie Yu
- Yunhan Luo
- Ani McCaskill
- Owen Sweetman

## Part 1: First ML model using Numpy
(3%) The implementation (using NumPy) of your first ML model as a function call to the cleaned data


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("csv_files/cleaned_video_data.csv")
df.head()

,video_id,title,published,views,likes,comments,duration,tags
0,9oYHU1hdDog,"Agents, AI & The Next Wave: Mike Clark on Vert...",2025-11-21T00:00:10Z,94,7,2,1602.0,"['Google', 'developers', 'pr_pr: Google Develo..."
1,9EGtawwvlNs,Build an AI Agent with Gemini 3,2025-11-20T21:45:49Z,5454,421,24,383.0,"['Google', 'developers', 'pr_pr: AI DevRel (fk..."
2,sCXuCczPE8A,What's on the newsdesk for Google Developers t...,2025-11-20T20:00:55Z,870,13,1,34.0,"['Google', 'developers', 'pr_pr: Google for De..."
3,hunRGj4Bpb0,Keep your code fresh and clean with YAGNI!,2025-11-20T19:30:53Z,963,20,1,34.0,"['Google', 'developers', 'pr_pr: Google for De..."
4,L0QHMLx02l4,"Introducing ADK Go, User Simulator, and more! ...",2025-11-20T17:02:19Z,2007,87,3,132.0,"['Google', 'developers', 'pr_pr: Google for De..."


#### Data Preparation

In [38]:
# Calculate engagement rate if not already present
if 'engagement_rate' not in df.columns:
    df['engagement_rate'] = (df['likes'] + df['comments']) / df['views']

# Remove invalid/null values
df = df.dropna(subset=['duration', 'engagement_rate', 'views', 'likes', 'comments'])
df = df[df['views'] > 0]

# Remove extreme outliers
df = df[df['duration'] <= 1000]  # Remove long videos
df = df[df['engagement_rate'] <= 1]  # Remove unrealistic rates

### Linear Regression Attempt For All Features

In [46]:
def evaluate_model(X, y):
    """
    Implement linear regression using NumPy
    Uses Normal Equation: theta = (X^T X)^(-1) X^T y
    """
    # Add bias column
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    
    # Normal equation
    theta = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
    
    return theta

def predict(X, theta):
    # Make predictions
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    return X_b.dot(theta)

def calculate_metrics(y_true, y_pred):
    #Calculate MSE and R^2
    mse = np.mean((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    ss_res = np.sum((y_true - y_pred) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    return mse, r2

# Define features and target
features = ['duration', 'views', 'likes', 'comments']
X = df[features].values
y = df['engagement_rate'].values

# Train model
theta = evaluate_model(X, y)
y_pred = predict(X, theta)

# Calculate metrics
mse, r2 = calculate_metrics(y, y_pred)

print(f"MSE: {mse:.6f}")
print(f"R²: {r2:.6f}")

MSE: 0.000053
R²: 0.671543


### Linear Regression Attempt for Indiviual Features

In [50]:
# MODEL 2: LINEAR REGRESSION - DURATION ONLY

X_duration = df[['duration']].values
theta_duration = manual_linear_regression(X_duration, y)
y_pred_duration = predict(X_duration, theta_duration)
mse_duration, r2_duration = calculate_metrics(y, y_pred_duration)

print("\nModel 2: Duration Only")
print(f"MSE: {mse_duration:.6f}")
print(f"R²: {r2_duration:.6f}")

# ============================================
# MODEL 3: LINEAR REGRESSION - VIEWS ONLY
# ============================================

X_views = df[['views']].values
theta_views = manual_linear_regression(X_views, y)
y_pred_views = predict(X_views, theta_views)
mse_views, r2_views = calculate_metrics(y, y_pred_views)

print("\nModel 3: Views Only")
print(f"MSE: {mse_views:.6f}")
print(f"R²: {r2_views:.6f}")

# ============================================
# MODEL 4: LINEAR REGRESSION - LIKES ONLY
# ============================================

X_likes = df[['likes']].values
theta_likes = manual_linear_regression(X_likes, y)
y_pred_likes = predict(X_likes, theta_likes)
mse_likes, r2_likes = calculate_metrics(y, y_pred_likes)

print("\nModel 4: Likes Only")
print(f"MSE: {mse_likes:.6f}")
print(f"R²: {r2_likes:.6f}")

NameError: name 'manual_linear_regression' is not defined